# INFO2950 Final Project

## Background Information

The pursuit of a college education has consistently been tied to graduates' financial success compared to non-graduates; on average, graduates are 1/2 as likely to be unemployed and have 84% higher income than those whose highest education is a high school diploma. Every year, high school graduates have to decide if the cost of college -- both tuition _and_ being out of the workforce for 4 years -- is worth the financial benefit. On top of the decision to attend, they must choose a major, a decision that could determine the degree of their financial success and the magnitude of benefits reaped from attending college.

However, this relationship is highly generalized. Not every degree concentration yields the same average financial success as others. For example, a degree in computer science or another form of engineering leads to a job market that has, on average, higher salaries than a degree in English.

A typical reason why a certain major's salary would be important to a college graduate is that many come to college for upward mobility. College is seen by many as a pathway to a higher financial and social standing as well as a higher quality of life. This extends further than simply their post-graduation salary, but the mean income of the location in which they live, as quality of life is largely affected by the wealth of those around you, as it impacts the opportunities available, public school funding, health, and much more. Perhaps the mean income of postgraduates' region of residence indicates the degree of their major's impact on their financial success and quality of life.

Thus, we are interested in exploring the relationship between the college major of a college postgraduate and the wealth of their county of residence. We decided we were most interested in exploring this relationship within New York State, where there is a _dramatic_ wealth gap between its most affluent regions (for example, New York City) versus its most poor regions; in fact, New York was recognized as the state with the highest gap between the rich and the poor, with a Gini coefficient score in 2022 of 0.52.


## Research Question

Question: Can we reliably predict a New York county's wealth based on their percentage of postgraduates of a certain age and major?

At the conclusion of this project, we seek to determine if there is a correlation between college major and the per capita income of the postgraduate county of residence in New York, and age and the per capita income of the postgraduate county of residence in New York. We will train a multivariable regression on data tables from both Wikepedia and the U.S. Census Bureau to see if we can reliably predict a county's per capita income given the population percentages of Science and Engineering, Business, Education, and Arts, Humanities, and Others for each of these age groups: 25-39 years, 40-64 years, and 65 years and over. Essentially, we aim to determine if college major and age affects the average wealth of where people live.

We defined county wealth using per capita income of the county and we solely considered New York counties for our regression. We defined postgraduates as people holding a Bachelor's degree who is at least 25 years old. 

## Original Data Description

We have two data tables from the U.S. Census Bureau and Wikepedia. 

Our data from Wikepedia:
https://en.wikipedia.org/wiki/List_of_New_York_locations_by_per_capita_income.

- Each row (observation) is a New York State county
- There are two columns (attributes) and they are: the county name and the county's per capita income. Before we copy-pasted it, the original dataframe on the website had additional columns for median household income, median family income, population, and number of households. We omitted these columns because they did not serve a purpose in our regression
- This dataset was created by Wikepedia to inform the public of the income per capita of each county in New York State
- The data is originally from the 2010 United States Census Data and the 2006-2010 American Community Survey 5-Year Estimates. The data from the Census is created, funded, and collected by the United States government for the purposes of better understanding the demographics of the country and informing policy
- Since this was a survey, the process of a voluntary survey may have influenced which data was actually gathered, since some may have declined to share their data or others may not have been within reach of or had access to the survey
- Those who participated were aware their data was going to be used for analyzing demographics
- We collected this data through copying and pasting the data into an Excel sheet and downloading it as a .csv file

Our data from the U.S. Census Bureau: https://data.census.gov/table/ACSST1Y2022.S1502?g=040XX00US36,36 using New York State (as a whole) and all New York Counties as filters for their data set, created in 2022.

- The rows (observations) each represented one person surveyed, categorized by age range and college major.
- The columns (attributes) are described below:
    - The first column houses the labels of groups of people by age and college major; the following columns pre- data cleaning have each county and the general state's estimate of people falling into each category, the margin of error, percent estimate, percent error, male estimate, male margin of error, male percent estimate, male percent margin of error, female estimate, female margin of error, female percent estimate, and female percent margin of error
- This dataset was created to gather information about the residents of the United States to better understand the demographics of the country and created more informed policy
- This data was created, funded, and collected by the government of the United States as a part of the Census, a large survey taken every 10 years
- Those who participated in the Census were aware that their data was being used to better understand demographics
- Since this was a survey, the process of a voluntary survey may have influenced which data was actually gathered, since some may have declined to share their data or been without access to complete the survey
- There are 5 rows and 469 columns, full of numerical data. No information is missing and the data is self-contained. The data identifies subpopulations by age and gender. Age is used to further divide the groups of majors and gender is used to divide each county's data
- This data was accessed using filters to narrow down the dataset to the variables: age, college major, and county

## Data Cleaning and Collection

### Importing packages

In [1109]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

### Data cleaning: selecting and extracting the relevant columns of U.S. Census Bureau data table and renaming columns

Read the .csv file:

In [1110]:
census_majors = pd.read_csv("nys_counties_college.csv")
census_majors.columns = census_majors.columns.str.replace(
    "    ", "")

Below, we see that there are many columns that are not relevant to our research question. We are only interested in each county's percent estimate column.

In [1111]:
print(census_majors.head())

                                    Label (Grouping)  \
0  Total population 25 years and over with a Bach...   
1                            Science and Engineering   
2             Science and Engineering Related Fields   
3                                           Business   
4                                          Education   

  New York!!Total!!Estimate New York!!Total!!Margin of Error  \
0                 5,556,561                          ±33,405   
1                 1,951,557                          ±21,418   
2                   513,403                          ±13,232   
3                 1,036,287                          ±19,358   
4                   560,313                          ±13,635   

  New York!!Percent!!Estimate New York!!Percent!!Margin of Error  \
0                         (X)                                (X)   
1                      35.10%                               ±0.4   
2                       9.20%                               ±0.2   
3     

Below, we are replacing the !!s and the spaces with an underscore, then selecting the relevant columns which have each county's total percentage (not including the columns regarding margin or error, total estimate, NYS as a whole, or Male/Female specific columns). There were a lot of columns to be dropped (more than half), so after trying to use .drop(), a TA helped us realize we should just extract the columns of interest and put it into a new dataframe, which we called census_majors_cleaned. We then rename the suffix of each column, _New_York_Percent_Estimate, to County so that we have [name] County as each column's name, so the column names are consistent with the County column of the Wikepedia dataframe.

In [1112]:
census_majors.columns = census_majors.columns.str.replace(
    "!!", "_")
census_majors.columns = census_majors.columns.str.replace(
    " ", "_")
#Checking that the column names are correct
display(census_majors)

,Label_(Grouping),New_York_Total_Estimate,New_York_Total_Margin_of_Error,New_York_Percent_Estimate,New_York_Percent_Margin_of_Error,New_York_Male_Estimate,New_York_Male_Margin_of_Error,New_York_Percent_Male_Estimate,New_York_Percent_Male_Margin_of_Error,New_York_Female_Estimate,...,"Westchester_County,_New_York_Percent_Estimate","Westchester_County,_New_York_Percent_Margin_of_Error","Westchester_County,_New_York_Male_Estimate","Westchester_County,_New_York_Male_Margin_of_Error","Westchester_County,_New_York_Percent_Male_Estimate","Westchester_County,_New_York_Percent_Male_Margin_of_Error","Westchester_County,_New_York_Female_Estimate","Westchester_County,_New_York_Female_Margin_of_Error","Westchester_County,_New_York_Percent_Female_Estimate","Westchester_County,_New_York_Percent_Female_Margin_of_Error"
0,Total population 25 years and over with a Bach...,"5,556,561","±33,405",(X),(X),"2,561,363","±21,001",(X),(X),"2,995,198",...,(X),(X),"168,019","±4,967",(X),(X),"197,801","±5,213",(X),(X)
1,Science and Engineering,"1,951,557","±21,418",35.10%,±0.4,"1,085,529","±16,174",42.40%,±0.6,"866,028",...,36.60%,±1.5,"70,551","±3,957",42.00%,±2.4,"63,221","±4,230",32.00%,±2.0
2,Science and Engineering Related Fields,"513,403","±13,232",9.20%,±0.2,"138,737","±6,246",5.40%,±0.2,"374,666",...,8.20%,±0.9,"7,169","±1,521",4.30%,±0.9,"22,775","±2,965",11.50%,±1.4
3,Business,"1,036,287","±19,358",18.60%,±0.3,"556,473","±13,673",21.70%,±0.5,"479,814",...,19.10%,±1.1,"37,741","±3,503",22.50%,±1.8,"32,236","±2,879",16.30%,±1.4
4,Education,"560,313","±13,635",10.10%,±0.2,"128,090","±6,084",5.00%,±0.2,"432,223",...,8.10%,±0.8,"4,763","±1,157",2.80%,±0.7,"24,851","±2,323",12.60%,±1.2
5,"Arts, Humanities and Others","1,495,001","±23,801",26.90%,±0.4,"652,534","±15,181",25.50%,±0.5,"842,467",...,28.00%,±1.3,"47,795","±3,654",28.40%,±1.9,"54,718","±4,107",27.70%,±1.9
6,DETAILED AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,25 to 39 years,"2,014,094","±21,802",(X),(X),"921,551","±14,714",(X),(X),"1,092,543",...,(X),(X),"44,714","±3,439",(X),(X),"50,929","±3,194",(X),(X)
8,Science and Engineering,"768,173","±14,817",38.10%,±0.7,"397,981","±10,089",43.20%,±1.0,"370,192",...,39.60%,±3.3,"18,701","±2,488",41.80%,±4.5,"19,139","±2,610",37.60%,±4.3
9,Science and Engineering Related Fields,"187,112","±7,607",9.30%,±0.4,"53,189","±4,136",5.80%,±0.4,"133,923",...,8.80%,±2.0,"2,399",±924,5.40%,±2.0,"6,049","±1,452",11.90%,±2.8


In [1113]:
census_majors_cleaned = census_majors[[
'Label_(Grouping)', 'Albany_County,_New_York_Percent_Estimate',
'Bronx_County,_New_York_Percent_Estimate',
'Broome_County,_New_York_Percent_Estimate',
'Cattaraugus_County,_New_York_Percent_Estimate',
'Cayuga_County,_New_York_Percent_Estimate',
'Chautauqua_County,_New_York_Percent_Estimate',
'Chemung_County,_New_York_Percent_Estimate',
'Clinton_County,_New_York_Percent_Estimate',
'Dutchess_County,_New_York_Percent_Estimate',
'Erie_County,_New_York_Percent_Estimate',
'Jefferson_County,_New_York_Percent_Estimate',
'Kings_County,_New_York_Percent_Estimate',
'Madison_County,_New_York_Percent_Estimate',
'Monroe_County,_New_York_Percent_Estimate',
'Nassau_County,_New_York_Percent_Estimate',
'New_York_County,_New_York_Percent_Estimate',
'Niagara_County,_New_York_Percent_Estimate',
'Oneida_County,_New_York_Percent_Estimate',
'Onondaga_County,_New_York_Percent_Estimate',
'Ontario_County,_New_York_Percent_Estimate',
'Orange_County,_New_York_Percent_Estimate',
'Oswego_County,_New_York_Percent_Estimate',
'Putnam_County,_New_York_Percent_Estimate',
'Queens_County,_New_York_Percent_Estimate',
'Rensselaer_County,_New_York_Percent_Estimate',
'Richmond_County,_New_York_Percent_Estimate',
'Rockland_County,_New_York_Percent_Estimate',
'St._Lawrence_County,_New_York_Percent_Estimate',
'Saratoga_County,_New_York_Percent_Estimate',
'Schenectady_County,_New_York_Percent_Estimate',
'Steuben_County,_New_York_Percent_Estimate',
'Suffolk_County,_New_York_Percent_Estimate',
'Sullivan_County,_New_York_Percent_Estimate',
'Tompkins_County,_New_York_Percent_Estimate',
'Ulster_County,_New_York_Percent_Estimate',
'Warren_County,_New_York_Percent_Estimate',
'Wayne_County,_New_York_Percent_Estimate',
'Westchester_County,_New_York_Percent_Estimate',
]]
#Confidence check: does the dataframe have the columns we want?
display(census_majors_cleaned)

,Label_(Grouping),"Albany_County,_New_York_Percent_Estimate","Bronx_County,_New_York_Percent_Estimate","Broome_County,_New_York_Percent_Estimate","Cattaraugus_County,_New_York_Percent_Estimate","Cayuga_County,_New_York_Percent_Estimate","Chautauqua_County,_New_York_Percent_Estimate","Chemung_County,_New_York_Percent_Estimate","Clinton_County,_New_York_Percent_Estimate","Dutchess_County,_New_York_Percent_Estimate",...,"Saratoga_County,_New_York_Percent_Estimate","Schenectady_County,_New_York_Percent_Estimate","Steuben_County,_New_York_Percent_Estimate","Suffolk_County,_New_York_Percent_Estimate","Sullivan_County,_New_York_Percent_Estimate","Tompkins_County,_New_York_Percent_Estimate","Ulster_County,_New_York_Percent_Estimate","Warren_County,_New_York_Percent_Estimate","Wayne_County,_New_York_Percent_Estimate","Westchester_County,_New_York_Percent_Estimate"
0,Total population 25 years and over with a Bach...,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,Science and Engineering,42.80%,32.90%,39.50%,34.50%,33.60%,30.70%,32.00%,26.30%,34.50%,...,43.00%,39.40%,36.20%,31.50%,31.70%,51.10%,29.50%,32.60%,39.20%,36.60%
2,Science and Engineering Related Fields,8.40%,11.70%,8.80%,8.30%,7.30%,14.00%,10.40%,12.10%,7.80%,...,7.60%,8.00%,11.40%,11.10%,7.70%,4.40%,5.50%,10.10%,11.00%,8.20%
3,Business,14.30%,19.70%,16.20%,16.60%,17.50%,11.80%,14.30%,14.30%,15.70%,...,17.20%,18.10%,14.90%,21.70%,11.20%,8.30%,16.10%,14.10%,17.80%,19.10%
4,Education,11.30%,10.50%,13.50%,20.40%,19.10%,19.50%,17.80%,22.50%,11.90%,...,11.00%,11.40%,15.40%,13.80%,15.30%,5.80%,12.10%,22.20%,15.10%,8.10%
5,"Arts, Humanities and Others",23.20%,25.20%,22.00%,20.20%,22.40%,24.00%,25.50%,24.80%,30.10%,...,21.30%,23.10%,22.10%,21.90%,34.10%,30.40%,36.90%,21.00%,17.00%,28.00%
6,DETAILED AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,25 to 39 years,(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
8,Science and Engineering,47.00%,37.10%,43.00%,36.10%,34.60%,22.90%,45.00%,23.30%,38.70%,...,43.40%,48.80%,37.30%,34.90%,39.80%,55.50%,35.40%,36.30%,45.20%,39.60%
9,Science and Engineering Related Fields,9.90%,11.70%,9.20%,7.10%,5.80%,12.10%,14.00%,18.20%,8.90%,...,10.80%,5.50%,10.60%,12.60%,5.30%,1.90%,8.00%,18.20%,15.10%,8.80%


In [1114]:
#Getting rid of added title (we just want the county name + "County")
census_majors_cleaned.columns = census_majors_cleaned.columns.str.replace(
    ",_New_York_Percent_Estimate", "")
#Changing the _s to a space for more familiar names 
#(underscore is less familiar to us)
census_majors_cleaned.columns = census_majors_cleaned.columns.str.replace(
    "_", " ")
#Setting the index to be the majors so when we transpose the dataframe,
#the columns will be named with the majors
census_majors_cleaned.set_index('Label (Grouping)', inplace=True)
#Checking to make sure the dataframe has the index we want
display(census_majors_cleaned)

,Albany County,Bronx County,Broome County,Cattaraugus County,Cayuga County,Chautauqua County,Chemung County,Clinton County,Dutchess County,Erie County,...,Saratoga County,Schenectady County,Steuben County,Suffolk County,Sullivan County,Tompkins County,Ulster County,Warren County,Wayne County,Westchester County
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Total population 25 years and over with a Bachelor's degree or higher,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
Science and Engineering,42.80%,32.90%,39.50%,34.50%,33.60%,30.70%,32.00%,26.30%,34.50%,32.20%,...,43.00%,39.40%,36.20%,31.50%,31.70%,51.10%,29.50%,32.60%,39.20%,36.60%
Science and Engineering Related Fields,8.40%,11.70%,8.80%,8.30%,7.30%,14.00%,10.40%,12.10%,7.80%,11.40%,...,7.60%,8.00%,11.40%,11.10%,7.70%,4.40%,5.50%,10.10%,11.00%,8.20%
Business,14.30%,19.70%,16.20%,16.60%,17.50%,11.80%,14.30%,14.30%,15.70%,19.90%,...,17.20%,18.10%,14.90%,21.70%,11.20%,8.30%,16.10%,14.10%,17.80%,19.10%
Education,11.30%,10.50%,13.50%,20.40%,19.10%,19.50%,17.80%,22.50%,11.90%,12.70%,...,11.00%,11.40%,15.40%,13.80%,15.30%,5.80%,12.10%,22.20%,15.10%,8.10%
"Arts, Humanities and Others",23.20%,25.20%,22.00%,20.20%,22.40%,24.00%,25.50%,24.80%,30.10%,23.70%,...,21.30%,23.10%,22.10%,21.90%,34.10%,30.40%,36.90%,21.00%,17.00%,28.00%
DETAILED AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25 to 39 years,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
Science and Engineering,47.00%,37.10%,43.00%,36.10%,34.60%,22.90%,45.00%,23.30%,38.70%,36.40%,...,43.40%,48.80%,37.30%,34.90%,39.80%,55.50%,35.40%,36.30%,45.20%,39.60%


We want to merge this data with the Wikepedia data, so we will align the county names as rows because a taller dataframe is better for regressions. We will use .transpose() to switch the columns and the rows of the cleaned census data. After performing that, we realized we have to set the index to our "Label (Grouping)" column, so we used set_index() above the transpose() command (in the cell above) so we could set the right column to be the index

In [1115]:
census_majors_cleaned = census_majors_cleaned.transpose()
display(census_majors_cleaned)

Label (Grouping),Total population 25 years and over with a Bachelor's degree or higher,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",DETAILED AGE,25 to 39 years,Science and Engineering,Science and Engineering Related Fields,...,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",65 years and over,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others"
Albany County,(X),42.80%,8.40%,14.30%,11.30%,23.20%,NaN,(X),47.00%,9.90%,...,7.70%,15.00%,11.00%,23.70%,(X),36.50%,7.30%,16.80%,21.60%,17.80%
Bronx County,(X),32.90%,11.70%,19.70%,10.50%,25.20%,NaN,(X),37.10%,11.70%,...,11.50%,21.20%,11.80%,25.00%,(X),29.10%,12.00%,13.10%,21.40%,24.40%
Broome County,(X),39.50%,8.80%,16.20%,13.50%,22.00%,NaN,(X),43.00%,9.20%,...,6.60%,22.00%,13.40%,20.70%,(X),38.70%,11.90%,11.90%,21.70%,15.70%
Cattaraugus County,(X),34.50%,8.30%,16.60%,20.40%,20.20%,NaN,(X),36.10%,7.10%,...,9.60%,18.50%,20.90%,19.00%,(X),36.80%,7.50%,17.20%,21.80%,16.70%
Cayuga County,(X),33.60%,7.30%,17.50%,19.10%,22.40%,NaN,(X),34.60%,5.80%,...,7.90%,22.80%,15.60%,16.10%,(X),26.20%,8.20%,13.00%,33.10%,19.50%
Chautauqua County,(X),30.70%,14.00%,11.80%,19.50%,24.00%,NaN,(X),22.90%,12.10%,...,13.00%,15.90%,17.20%,17.80%,(X),26.60%,17.50%,4.40%,26.20%,25.30%
Chemung County,(X),32.00%,10.40%,14.30%,17.80%,25.50%,NaN,(X),45.00%,14.00%,...,5.40%,22.20%,19.30%,27.10%,(X),27.40%,14.60%,14.60%,29.90%,13.50%
Clinton County,(X),26.30%,12.10%,14.30%,22.50%,24.80%,NaN,(X),23.30%,18.20%,...,9.20%,16.90%,18.10%,27.70%,(X),26.30%,10.60%,8.70%,33.60%,20.90%
Dutchess County,(X),34.50%,7.80%,15.70%,11.90%,30.10%,NaN,(X),38.70%,8.90%,...,6.20%,20.40%,11.20%,29.30%,(X),32.00%,9.30%,9.50%,21.70%,27.50%
Erie County,(X),32.20%,11.40%,19.90%,12.70%,23.70%,NaN,(X),36.40%,11.10%,...,11.50%,22.40%,12.20%,23.70%,(X),29.10%,12.00%,13.60%,22.70%,22.60%


Now we have to create an index for the rows, because we want the county to be a column rather than an index, since we cannot use a County index like we would with a column as a key when merging dataframes.

In [1116]:
census_majors_cleaned.reset_index(inplace=True)
census_majors_cleaned.rename(
    columns = {'index':'County'}, inplace = True)
display(census_majors_cleaned)

Label (Grouping),County,Total population 25 years and over with a Bachelor's degree or higher,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",DETAILED AGE,25 to 39 years,Science and Engineering,...,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",65 years and over,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others"
0,Albany County,(X),42.80%,8.40%,14.30%,11.30%,23.20%,NaN,(X),47.00%,...,7.70%,15.00%,11.00%,23.70%,(X),36.50%,7.30%,16.80%,21.60%,17.80%
1,Bronx County,(X),32.90%,11.70%,19.70%,10.50%,25.20%,NaN,(X),37.10%,...,11.50%,21.20%,11.80%,25.00%,(X),29.10%,12.00%,13.10%,21.40%,24.40%
2,Broome County,(X),39.50%,8.80%,16.20%,13.50%,22.00%,NaN,(X),43.00%,...,6.60%,22.00%,13.40%,20.70%,(X),38.70%,11.90%,11.90%,21.70%,15.70%
3,Cattaraugus County,(X),34.50%,8.30%,16.60%,20.40%,20.20%,NaN,(X),36.10%,...,9.60%,18.50%,20.90%,19.00%,(X),36.80%,7.50%,17.20%,21.80%,16.70%
4,Cayuga County,(X),33.60%,7.30%,17.50%,19.10%,22.40%,NaN,(X),34.60%,...,7.90%,22.80%,15.60%,16.10%,(X),26.20%,8.20%,13.00%,33.10%,19.50%
5,Chautauqua County,(X),30.70%,14.00%,11.80%,19.50%,24.00%,NaN,(X),22.90%,...,13.00%,15.90%,17.20%,17.80%,(X),26.60%,17.50%,4.40%,26.20%,25.30%
6,Chemung County,(X),32.00%,10.40%,14.30%,17.80%,25.50%,NaN,(X),45.00%,...,5.40%,22.20%,19.30%,27.10%,(X),27.40%,14.60%,14.60%,29.90%,13.50%
7,Clinton County,(X),26.30%,12.10%,14.30%,22.50%,24.80%,NaN,(X),23.30%,...,9.20%,16.90%,18.10%,27.70%,(X),26.30%,10.60%,8.70%,33.60%,20.90%
8,Dutchess County,(X),34.50%,7.80%,15.70%,11.90%,30.10%,NaN,(X),38.70%,...,6.20%,20.40%,11.20%,29.30%,(X),32.00%,9.30%,9.50%,21.70%,27.50%
9,Erie County,(X),32.20%,11.40%,19.90%,12.70%,23.70%,NaN,(X),36.40%,...,11.50%,22.40%,12.20%,23.70%,(X),29.10%,12.00%,13.60%,22.70%,22.60%


Now we are ready to split this data into other subgroups and merge that data with the Wikepedia data!
Note: we know we have NaNs in our dataframe, but we will solely be using this dataframe to extract columns from, so we will be "cleaning" the NaNs out by not using those columns (since the columns of NaNs serve to show which columns are for which age)

### Data collection: Excel and downloading the Wikepedia table

In [1117]:
wikepedia_url = "https://en.wikipedia.org/wiki/List_of_New_York_locations_by_per_capita_income"

In [ ]:
wikepedia_result = requests.get(wikepedia_url)

In [ ]:
with open("county_wealth.html", "w") as writer:
    writer.write(wikepedia_result.text)

In [ ]:
with open("county_wealth.html", "r") as reader:
    html_source = reader.read()

In [ ]:
# Confidence check
html_source[:20]

In [ ]:
page = BeautifulSoup(html_source, "html.parser")

Below, you will find that there are no classes in the < tr > tags that we scraped.

In [ ]:
wikepedia_county_income = page.find("table", 
                                    {"class": "wikitable"})
wikepedia_county_income = wikepedia_county_income.find_next(
    'tbody').find_all('tr')
print(wikepedia_county_income)

We attempted to data scrape the Wikepedia website, but we were unsuccessful because the < tr > tags that contained the data we were trying to scrape did not have a class associated with them, and TAs we consulted recommended manually pasting the data into an Excel spreadsheet and downloading it as a .csv file instead.

Thus, we manually copy-pasted the Wikepedia data into Excel and downloaded that file as a .csv

In [ ]:
county_wealth = pd.read_csv("county_wealth.csv")
display(county_wealth)

### Data Cleaning: Adding "County" to the county names in county_wealth

To perform an inner join on the tables, we have to rename each row of the "County" column of county_wealth, since the county name does not have "county" after it, as it does in census_majors' county columns.

In [ ]:
county_wealth['County'] = county_wealth['County'] + ' County'
county_wealth['County'].iloc[[0]] = 'New York County'
display(county_wealth)

We also have to change the Per capita income column values from strings to integers.

In [ ]:
#Get rid of the commas and extra spaces in the values 
#so they can be converted to integers
county_wealth['Per capita income'] = county_wealth[
    'Per capita income'].str.replace('  ,', '')
county_wealth['Per capita income'] = county_wealth[
    'Per capita income'].str.replace(', ', '')
#Convert to int
county_wealth['Per capita income'] = county_wealth[
    'Per capita income'].astype(int)
#Confidence check - print the type of the column 
#and display the dataframe
print(county_wealth['Per capita income'].dtypes)
display(county_wealth)

### Data cleaning: splitting census data

Here we are splitting the data into two subsets:
- One dataframe where the age is disregarded and all of the ages are compiled into one major, so that there is one value for each major (ageless_majors)
- Three dataframes where each age is displayed in full on its own (young_majors, middle_majors, old_majors)

After we create these, we display each dataframe and then merge it with the wikepedia dataframe. We want to merge this via a left join to combine the data unique to each individual dataframe being joined so we can look at specific age's majors' impact on county income per capita. 

In [ ]:
#create the census dataframe for all ages (columns 1 - 6)
only_census_total = census_majors_cleaned.iloc[:, 1: 7]
#Drop the NaN column from the only_census_total dataframe 
#to create the no-NaN ageless_majors dataframe
ageless_majors = only_census_total.drop(columns=[
"Total population 25 years and over with a Bachelor's degree or higher"])
#Add the County column
ageless_majors['County'] = census_majors_cleaned['County']
#Confidence check
display(ageless_majors)

In [ ]:
#create the census dataframe for only ages 25-39 
#(columns 9 - 13)
young_majors = census_majors_cleaned.iloc[:, 9:14]
#Add the County column
young_majors['County'] = census_majors_cleaned['County']
#Confidence check
display(young_majors)

In [ ]:
#create census dataframe only for ages 40-64 
#(columns 15 - 19)
middle_majors = census_majors_cleaned.iloc[:, 15:20]
#Add the County column
middle_majors['County'] = census_majors_cleaned[
    'County']
#Confidence check
display(middle_majors)

In [ ]:
#create census dataframe for only ages 65+ 
#(columns 21 - 25)
old_majors = census_majors_cleaned.iloc[:, 21:26]
#Add County column
old_majors['County'] = census_majors_cleaned[
    'County']
#Confidence check
display(old_majors)

### Data cleaning: merging dataframes

We want to merge our cleaned census dataframe with our Wikepedia dataframe for comparison purposes. We will use a LEFT JOIN on the county key, because both dataframes have a county column that we have ensured has shared values. The Wikipedia data has more counties in its County column than the census's County column, but the join will ensure that the resulting merged dataframes will only have the shared counties. We will do this for census_majors_cleaned (the cleaned version of the original census dataframe), ageless_majors, and the three age-specific dataframes (young_majors, middle_majors, and old_majors).

In [ ]:
#Merge the whole census_majors_cleaned 
#dataframe with the county_wealth (Wikepedia) dataframe
merged_df = county_wealth.merge(
    census_majors_cleaned, 
    left_on='County', right_on ='County')
display(merged_df)

Here we will merge the non-age-specific dataframe with the county_wealth dataframe.

In [ ]:
ageless_merged_df = county_wealth.merge(
    ageless_majors, left_on='County', 
    right_on ='County')
display(ageless_merged_df)

We also merged each of the age-specific data frames with the county_wealth dataframe.

In [ ]:
young_merged_df = county_wealth.merge(
    young_majors, left_on='County', 
    right_on ='County')
display(young_merged_df)

In [ ]:
middle_merged_df = county_wealth.merge(
    middle_majors, left_on='County', 
    right_on ='County')
display(middle_merged_df)

In [ ]:
old_merged_df = county_wealth.merge(
    old_majors, left_on='County', 
    right_on ='County')
display(old_merged_df)

Lastly, we will get rid of the percent signs for the county's college major data and convert the strings into floats. We will do this for every merged dataframe so that we can run regressions and compute functions on the percentage data.

We will start by getting rid of the %s in the ageless_majors dataframe by replacing the % for each column:

In [ ]:
ageless_merged_df['Science and Engineering'] = ageless_merged_df[
    'Science and Engineering'].str.replace('%', '')
ageless_merged_df['Science and Engineering'] = ageless_merged_df[
    'Science and Engineering'].astype(float)
ageless_merged_df['Science and Engineering Related Fields'
                 ] = ageless_merged_df[
    'Science and Engineering Related Fields'].str.replace('%', '')
ageless_merged_df['Science and Engineering Related Fields'
                 ] = ageless_merged_df[
    'Science and Engineering Related Fields'].astype(float)
ageless_merged_df['Business'] = ageless_merged_df[
    'Business'].str.replace('%', '')
ageless_merged_df['Business'] = ageless_merged_df[
    'Business'].astype(float)
ageless_merged_df['Education'] = ageless_merged_df[
    'Education'].str.replace('%', '')
ageless_merged_df['Education'] = ageless_merged_df[
    'Education'].astype(float)
ageless_merged_df['Arts, Humanities and Others'
] = ageless_merged_df['Arts, Humanities and Others'
                     ].str.replace('%', '')
ageless_merged_df['Arts, Humanities and Others'
] = ageless_merged_df['Arts, Humanities and Others'
                     ].astype(float)
#Confidence check
print(ageless_merged_df[
    'Science and Engineering'].dtypes)
display(ageless_merged_df)

And now we will do the same to the three age-divided dataframes by replacing the % for each column in each dataframe:

In [ ]:
young_merged_df['Science and Engineering'] = young_merged_df[
    'Science and Engineering'].str.replace('%', '')
young_merged_df['Science and Engineering'] = young_merged_df[
    'Science and Engineering'].astype(float)
young_merged_df[
    'Science and Engineering Related Fields'
] = young_merged_df[
    'Science and Engineering Related Fields'
].str.replace('%', '')
young_merged_df[
    'Science and Engineering Related Fields'
] = young_merged_df[
    'Science and Engineering Related Fields'
].astype(float)
young_merged_df['Business'] = young_merged_df[
    'Business'].str.replace('%', '')
young_merged_df['Business'] = young_merged_df[
    'Business'].astype(float)
young_merged_df['Education'] = young_merged_df[
    'Education'].str.replace('%', '')
young_merged_df['Education'] = young_merged_df[
    'Education'].astype(float)
young_merged_df['Arts, Humanities and Others'
] = young_merged_df['Arts, Humanities and Others'
].str.replace('%', '')
young_merged_df['Arts, Humanities and Others'
] = young_merged_df['Arts, Humanities and Others'
                   ].astype(float)
#Confidence check
print(young_merged_df['Science and Engineering'
                     ].dtypes)
display(young_merged_df)

In [ ]:
middle_merged_df['Science and Engineering'
] = middle_merged_df['Science and Engineering'
                    ].str.replace('%', '')
middle_merged_df['Science and Engineering'
] = middle_merged_df['Science and Engineering'
                    ].astype(float)
middle_merged_df['Science and Engineering Related Fields'
] = middle_merged_df['Science and Engineering Related Fields'
                    ].str.replace('%', '')
middle_merged_df['Science and Engineering Related Fields'
] = middle_merged_df['Science and Engineering Related Fields'
                    ].astype(float)
middle_merged_df['Business'
] = middle_merged_df['Business'].str.replace('%', '')
middle_merged_df['Business'
] = middle_merged_df['Business'].astype(float)
middle_merged_df['Education'
] = middle_merged_df['Education'].str.replace('%', '')
middle_merged_df['Education'
] = middle_merged_df['Education'].astype(float)
middle_merged_df['Arts, Humanities and Others'
] = middle_merged_df['Arts, Humanities and Others'
                    ].str.replace('%', '')
middle_merged_df['Arts, Humanities and Others'
] = middle_merged_df['Arts, Humanities and Others'
            ].astype(float)
#Confidence check
print(middle_merged_df['Science and Engineering'
                      ].dtypes)
display(middle_merged_df)

In [ ]:
old_merged_df['Science and Engineering'] = old_merged_df[
    'Science and Engineering'].str.replace('%', '')
old_merged_df['Science and Engineering'] = old_merged_df[
    'Science and Engineering'].astype(float)
old_merged_df['Science and Engineering Related Fields'
] = old_merged_df['Science and Engineering Related Fields'
                 ].str.replace('%', '')
old_merged_df['Science and Engineering Related Fields'
] = old_merged_df['Science and Engineering Related Fields'
                 ].astype(float)
old_merged_df['Business'
] = old_merged_df['Business'].str.replace('%', '')
old_merged_df['Business'] = old_merged_df[
'Business'].astype(float)
old_merged_df['Education'] = old_merged_df['Education'
                        ].str.replace('%', '')
old_merged_df['Education'] = old_merged_df[
    'Education'].astype(float)
old_merged_df['Arts, Humanities and Others'
] = old_merged_df['Arts, Humanities and Others'
                 ].str.replace('%', '')
old_merged_df['Arts, Humanities and Others'
] = old_merged_df['Arts, Humanities and Others'
                 ].astype(float)
#Confidence check
print(old_merged_df['Science and Engineering'
                   ].dtypes)
display(old_merged_df)

Now the merged dataframes are ready for our exploratory data analysis!

## Exploratory Data Analysis

We will begin by performing general analysis and visualization on the larger, more overarching index_county dataframe. 

We want to find the values of:
- Mean of each major's percentage within all counties
- Median of each major's percentage within all counties
- Mean of each major's percentage within all counties for each age
- Median of each major's percentage within all counties for each age

We want to display contextual visualizations to demonstrate:
- Each county's per capita income
- A comparison of counties' science and engineering majors percentage for all ages
- A comparison of counties' science and engineering related majors percentage for all ages
- A comparison of counties' business majors percentage for all ages
- A comparison of counties' education majors percentage for all ages
- A comparison of counties' arts, humanities, and others majors percentage for all ages

Begin with the mean and median values for ageless_merged_df:

In [ ]:
print('Mean of Science and Engineering percentage \
within all counties: ' +
str(np.round(np.mean(ageless_merged_df['Science and \
Engineering']),2)))
print(
'Mean of Science and Engineering Related \
Fields percentage within all counties: ' 
+ str(np.round(np.mean(ageless_merged_df[
    'Science and Engineering Related Fields']),2)))
print('Mean of Business percentage within all counties: ' + 
str(np.round(np.mean(ageless_merged_df['Business']),2)))
print('Mean of Education percentage within all counties: ' + 
str(np.round(np.mean(ageless_merged_df['Education']),2)))
print('Mean of Arts, Humanities and Others percentage \
within all counties: ' + 
str(np.round(np.mean(ageless_merged_df['Arts, \
Humanities and Others']),2)))

In [ ]:
print('Median of Science and Engineering percentage \
within all counties: ' + 
str(np.round(np.median(ageless_merged_df['Science \
and Engineering']),2)))
print('Median of Science and Engineering Related \
Fields percentage within all counties: ' 
+ str(np.round(np.median(ageless_merged_df['Science \
and Engineering Related Fields']),2)))
print('Median of Business percentage within all counties: ' + 
str(np.round(np.median(ageless_merged_df['Business']),2)))
print('Median of Education percentage within all counties: ' +
str(np.round(np.median(ageless_merged_df['Education']),2)))
print('Median of Arts, Humanities and Others percentage \
within all counties: ' + 
str(np.round(np.median(ageless_merged_df['Arts, Humanities \
and Others']),2)))

Now the mean and median for each age range: 

For young_merged_df (ages 25-39):

In [ ]:
print('Mean of Science and Engineering percentage within all \
counties (ages 25-39): '
+ str(np.round(np.mean(young_merged_df['Science and Engineering']),2)))
print('Mean of Science and Engineering Related Fields percentage \
within all counties (ages 25-39): ' 
+ str(np.round(np.mean(young_merged_df['Science and Engineering \
Related Fields']),2)))
print('Mean of Business percentage within all counties (ages 25-39): ' + 
str(np.round(np.mean(young_merged_df['Business']),2)))
print('Mean of Education percentage within all counties (ages 25-39): ' + 
str(np.round(np.mean(young_merged_df['Education']),2)))
print('Mean of Arts, Humanities and Others percentage within all \
counties (ages 25-39): ' + 
str(np.round(np.mean(young_merged_df['Arts, Humanities and Others']),2)))

In [ ]:
print('Median of Science and Engineering percentage within all \
counties (ages 25-39): ' 
+ str(np.round(np.median(young_merged_df['Science and Engineering']),2)))
print('Median of Science and Engineering Related Fields percentage \
within all counties (ages 25-39): ' 
+ str(np.round(np.median(young_merged_df['Science and Engineering \
Related Fields']),2)))
print('Median of Business percentage within all counties (ages 25-39): ' 
+ str(np.round(np.median(young_merged_df['Business']),2)))
print('Median of Education percentage within all counties (ages 25-39): '
+ str(np.round(np.median(young_merged_df['Education']),2)))
print('Median of Arts, Humanities and Others percentage within all \
counties (ages 25-39): '
+ str(np.round(np.median(young_merged_df['Arts, Humanities and Others']),2)))

For middle_merged_df (ages 40-64):

In [ ]:
print('Mean of Science and Engineering percentage within all \
counties (ages 40-64): ' + 
str(np.round(np.mean(middle_merged_df['Science and Engineering']),2)))
print('Mean of Science and Engineering Related Fields percentage \
within all counties (ages 40-64): ' 
+ str(np.round(np.mean(middle_merged_df['Science and Engineering \
Related Fields']),2)))
print('Mean of Business percentage within all counties (ages 40-64): ' 
+ str(np.round(np.mean(middle_merged_df['Business']),2)))
print('Mean of Education percentage within all counties (ages 40-64): ' 
+ str(np.round(np.mean(middle_merged_df['Education']),2)))
print('Mean of Arts, Humanities and Others percentage within all \
counties (ages 40-64): ' 
+ str(np.round(np.mean(middle_merged_df['Arts, Humanities and Others']),2)))

In [ ]:
print('Median of Science and Engineering percentage within all \
counties (ages 40-64): ' 
+ str(np.round(np.median(middle_merged_df['Science and Engineering']),2)))
print('Median of Science and Engineering Related Fields percentage \
within all counties (ages 40-64): ' 
+ str(np.round(np.median(middle_merged_df['Science and Engineering \
Related Fields']),2)))
print('Median of Business percentage within all counties (ages 40-64): ' 
+ str(np.round(np.median(middle_merged_df['Business']),2)))
print('Median of Education percentage within all counties (ages 40-64): '
+ str(np.round(np.median(middle_merged_df['Education']),2)))
print('Median of Arts, Humanities and Others percentage within all \
counties (ages 40-64): ' 
+ str(np.round(np.median(middle_merged_df['Arts, Humanities and Others']),2)))

For old_merged_df (ages 65+):

In [ ]:
print('Mean of Science and Engineering percentage \
within all counties (ages 65+): ' 
+ str(np.round(np.mean(old_merged_df['Science and Engineering']),2)))
print('Mean of Science and Engineering Related Fields percentage \
within all counties (ages 65+): ' 
+ str(np.round(np.mean(old_merged_df['Science and \
Engineering Related Fields']),2)))
print('Mean of Business percentage within all \
counties (ages 65+): ' 
+ str(np.round(np.mean(old_merged_df['Business']),2)))
print('Mean of Education percentage within all \
counties (ages 65+): ' 
+ str(np.round(np.mean(old_merged_df['Education']),2)))
print('Mean of Arts, Humanities and Others \
percentage within all counties (ages 65+): ' 
+ str(np.round(np.mean(old_merged_df['Arts, \
Humanities and Others']),2)))

Start by displaying per capita income for each county (ranked) to understand the general differences between the rankings

In [ ]:
#display general trend of each county (ranked by per capita income) 
#vs. their per capita income
general_graph = plt.bar(ageless_merged_df['County'], 
        ageless_merged_df['Per capita income'])
plt.title("NYS Counties' per capita income")
plt.xlabel("County name, ranked by per capita income")
plt.ylabel("Per capita income ($)")
plt.xticks(rotation=85)
display(general_graph)

This visualization is particularly interesting, it seems as though New York County and Westchester County have dramatically higher per capita incomes than the rest of the state's counties, which have relatively similar per capita incomes approximately ranging from 40,000 dollars to  20,000 dollars.

Now we will display each county's percentage of Science and Engineering majors relative to the other counties:

In [ ]:
science_engineering_barplot_ageless = sns.barplot(ageless_merged_df,
    x="County", y="Science \
and Engineering")
science_engineering_barplot_ageless.set_xticklabels(
    ageless_merged_df['County'],rotation=85)
science_engineering_barplot_ageless.set(
    xlabel="County name, ranked by per capita income", 
    ylabel='Percentage of Science and Engineering', 
    title='Percentage of Science and Engineering \
majors for each county')
display(science_engineering_barplot_ageless)

Tompkins County clearly has the greatest number of Science and Engineering graduates with a percentage greater than 50, while other counties fall into the range of roughly 40% to 25%. This is interesting, since Tompkins County is roughly in the middle tier of per capita income, so such a large percentage of Science and Engineering graduates is noteable.

Now we will display each county's percentage of Science and Engineering Related Fields majors relative to the other counties:

In [ ]:
science_engineering_barplot_related_ageless = sns.barplot(
    ageless_merged_df, x="County", 
    y="Science and Engineering Related Fields")
science_engineering_barplot_related_ageless.set_xticklabels(
    ageless_merged_df['County'],rotation=85)
science_engineering_barplot_related_ageless.set(
    xlabel="County name, ranked by per capita income", 
    ylabel='Percentage of Science and Engineering Related Fields', 
    title='Percentage of Science and Engineering Related Fields \
majors for each county')
display(science_engineering_barplot_related_ageless)

Overall, the category of Science and Engineering Related Fields makes up a very low percentage within each county, with its highest percentage of a county's graduates being 14% in Ontario County, Oneida County, and Chautauqua County, and its lowest being roughly 4% in New York County and in Tompkins County, despite the two counties scoring at the high end of the Science and Engineering category. Perhaps this is due to the way the data was categorized into either the Science and Engineering category or the Science and Engineering _Related_ Fields category.

Now we will display each county's percentage of Business majors relative to the other counties:

In [ ]:
business_barplot_ageless = sns.barplot(
    ageless_merged_df, x="County", y="Education")
business_barplot_ageless.set_xticklabels(
    ageless_merged_df['County'],rotation=85)
business_barplot_ageless.set(
    xlabel="County name, ranked by per capita income", 
    ylabel='Percentage of Business', 
    title='Percentage of Business majors for each county')
display(business_barplot_ageless)

There seems to be a general positive relationship between the percentage of business majors and the per capita income of a county; as per capita income decreases, the percentage of business majors seems to decrease overall.

Now we will display each county's percentage of Education majors relative to the other counties:

In [ ]:
education_barplot_ageless = sns.barplot(
    ageless_merged_df, x="County", y="Education")
education_barplot_ageless.set_xticklabels(
    ageless_merged_df['County'],rotation=85)
education_barplot_ageless.set(
    xlabel="County name, ranked by per capita income", 
    ylabel='Percentage of Education', 
    title='Percentage of Education majors for each county')
display(education_barplot_ageless)

The overall trend viewable in the above visualization is: as per capita income decreases, the percentage of Education degree-holders increases. An interesting outlier is Warren County, which has a much higher percentage of Education majors relative to similarly-wealthy counties. Upon inspection, the county is located very close to Albany, the capital of the state and has a sizeable population, so it may be that many educators of the metropolis live in the county. However, this claim is purely speculative and requires further investigation to validate a cause for the county's higher percentage of Education majors.

Now we will display each county's percentage of Arts, Humanities and Others majors relative to the other counties:

In [ ]:
arts_barplot_ageless = sns.barplot(
ageless_merged_df, x="County", y="Arts, Humanities and Others")
arts_barplot_ageless.set_xticklabels(
    ageless_merged_df['County'],rotation=85)
arts_barplot_ageless.set(
    xlabel="County name, ranked by per capita income", 
    ylabel='Percentage of Arts, Humanities and Others', 
    title='Percentage of Arts, Humanities and Others majors \
for each county')
display(arts_barplot_ageless)

There does not seem to be any discernible relationship between the percentage of arts, humanities, and other majors and the county's per capita income.

## Data Description of Analysis-Ready data

- What are the observations (rows) and the attributes (columns)?
    - For the ageless_merged_df dataframe, it has 37 rows and 7 columns. Each row is a county (they are ranked by per capita income) and the first column is the county name. The rest of the columns are each group / category of college major. The values in those columns are the percentage of the specific major in a specific county.
    - For the young_merged_df dataframe, it has 37 rows and 7 rows, all of which are the same as the ageless_merged_df dataframe, but only pertaining to the values for participants who were between the ages of 25 and 39.
    - For the middle_merged_df dataframe, it has 37 rows and 7 rows, all of which are the same as the ageless_merged_df dataframe, but only pertaining to the values for participants who were between the ages of 40 and 64.
    - For the old_merged_df dataframe, it has 37 rows and 7 rows, all of which are the same as the ageless_merged_df dataframe, but only pertaining to the values for participants who were 65 years old and older.
- What preprocessing was done, and how did the data come to be in the form that you are using?
    - We had to grab the columns from the census data that we were interested in; originally, the data had a lot of columns unrelated to the major percentage of each county. We also had to convert many string columns into floats and ints, as the % signs and the commas in the data we extracted from the Census and Wikepedia both had numerical values in the form of a string. We also had to change the names of many columns because they had unnecessarily long and verbose names, including the County names. With our cleaned data, we divided that one big dataframe into three dataframes, each one representing one age group. We merged the Wikepedia data and the big cleaned Census data we had into one. We then merged the Wikepedia dataframe with each of the three smaller dataframes for each age group. We split it into age-specific dataframes because we wanted to view the data in both an age-specific context and an age-blind context.
- What do the instances that comprise the dataset represent?
    - The data in the major-name columns represent the percentage of people in that county who reported their major as being within that category of majors.
    - The data in the county name columns represents the county being measured.

## Preregistration Hypotheses

- Hypothesis 1: Do counties with more business majors than any other major tend to be richer than counties with a lower percentage of business majors? 

- $H_0$: Counties with more business majors than any other major do not have a significant difference in wealth compared to counties with a lower percentages of business majors.
- $H_A$: Counties with more business majors than any other major do have a significant difference in wealth compared to counties with a lower percentages of business majors.

We seek to determine whether a correlation exists between college major and the wealth of a graduate's county of residency. Majoring in business typically involves positions in finance, an industry that is assumed to be profitable and lucrative. By specifically investigating a potential relationship between business major percentage and a county's wealth, we can support or counter this assumption.

Using a linear regression, we will input business major percentage as a variable and output county wealth. We will use this model to determine the correlation coefficient between the input and the output. We will also create visual graphs to display the potential relationship between the input and the output.

- Hypothesis 2: Do science and engineering majors tend to reside in a county with a high per capita income? 

- $H_0$: Science and engineering majors are equally as likely as other majors to reside in a county with a high per capita income.
- $H_A$: Science and engineering majors are significantly more likely than other majors to reside in a county with a high per capita income.

Similar to business, science and engineering professions have been increasingly viewed as profitable due to the rise of the technology industry. We seek to determine whether a relationship exists between science and engineering majors and the wealth of the county in which they reside in. By determining the existence of such a relationship, we can either reinforce or contradict this assumption with evidence.

Similar to the previous hypothesis, we will use a linear regression to determine the correlation coefficient between the input and the output. We will also display visuals to determine the distribution of the data.

## Hypothesis 1 - Do counties with more business majors tend to be richer than counties with lower percentages of business majors?

Below, we are isolating the relevant columns of our ageless_merged_df dataframe and fitting a regression to it.

In [ ]:
hyp1_df = ageless_merged_df[['Per capita income',
                             'Science and Engineering', 
                             'Science and Engineering Related Fields', 
                             'Business', 'Education', 
                             'Arts, Humanities and Others']]
display(hyp1_df)

We are now isolating the relevant columns to Hypothesis 1, namely the percentage of business majors within each county (X) and the per capita income of each county (y):

In [ ]:
X = hyp1_df[['Business']]
y = hyp1_df[['Per capita income']]
plt.scatter(X, y, marker='o')
plt.xlabel('Percentage of business majors')
plt.ylabel('Per capita income')
plt.title('Business major percentage and per capita income of NYS counties')
plt.show()

It seems that there is a general trend of greater percentages of business majors within a county corresponding to a greater per capita income, but there are two large outliers.

Now we are creating our train set x's and y's as well as our test set x's and y's:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state=2950)


We want to display summary statistics of our X_train data:

In [ ]:
print(X_train.describe())

In [ ]:
h1_model = LinearRegression().fit(X_train, y_train)

In the above cell, we have trained a linear regression from a train test split to predict per capita income with the percentage of business majors in each county as the input. Below, we are making predictions of per capita income given business major percentage using the train set. This allows us to view our model's predictions of per capita income (h1_y_hat_train) to our _actual_ y values (y_train)

In [ ]:
h1_y_hat_train = h1_model.predict(X_train)

Now our model will use the X_&nbsp;_test_ data to predict y_test. This allows us to determine if the model works on "new" data (data it was not trained on), since the X_test data was not included in X_train. We will compare these predictions (predicted values of county per capita income) to y_test (the actual y values).

In [ ]:
h1_y_hat_test = h1_model.predict(X_test)

To determine how correct the model's predictions are (and how much we should trust it), we will now calculate its evaluation metrics, starting with the train set:

In [ ]:
mse = mean_squared_error(y_train, h1_y_hat_train)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, h1_y_hat_train)
mape = mean_absolute_percentage_error(y_train, h1_y_hat_train)
print('Mean squared error: ' + str(np.round(mse, 3)))
print('Root mean squared error: ' + str(np.round(rmse, 3)))
print('Mean absolute error: ' + str(np.round(mae, 3)))
print('Mean absolute percentage error: ' + str(np.round(mape, 3)))

And now we will do the same for the test set:

In [ ]:
mse = mean_squared_error(y_test, h1_y_hat_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, h1_y_hat_test)
mape = mean_absolute_percentage_error(y_test, h1_y_hat_test)
print('Mean squared error: ' + str(np.round(mse, 3)))
print('Root mean squared error: ' + str(np.round(rmse, 3)))
print('Mean absolute error: ' + str(np.round(mae, 3)))
print('Mean absolute percentage error: ' + str(np.round(mape, 3)))

From these evaluation metrics, it seems as though the model performs better on the test set than the train set due to the dramatically lower MSE value for the test set relative to the MSE for the train set. This is most likely a fluke test set performance. We will now iterate on our model to get better predictions.

## Hypothesis 2: Do science and engineering majors tend to reside in a county with a higher per capita income?

We will create a copy of the hyp1_df dataframe, and use it to isolate the relevant columns to Hypothesis 2, namely the percentage of science and engineering majors within each county (X) and the per capita income of each county (y):

In [ ]:
hyp2_df = hyp1_df

In [ ]:
X = hyp2_df[['Science and Engineering']]
y = hyp2_df[['Per capita income']]
plt.scatter(X, y, marker='o')
plt.xlabel('Percentage of science and engineering majors')
plt.ylabel('Per capita income')
plt.title('Science and Engineering major percentage and per capita income of NYS counties')
plt.show()

The majority of counties seem to have a percentage of science and engineering majors ranging from 30-40 percent, with the entire range being from roughly 25% to 53.. However, the range of per capita income is drastically greater, ranging from 20,000 dollars to greater than 100,000 dollars. The majority of counties fall between 30% to 40% science and engineering majors with 20,000 to 40,000 dollars in per capita income.

Now we are creating our train set x's and y's as well as our test set x's and y's to utilize a train-test-split on our data to be able to test our regression on "new" test data after training it:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state=2950)


We want to display summary statistics of our X_train data:

In [ ]:
print(X_train.describe())

In [ ]:
h2_model = LinearRegression().fit(X_train, y_train)

In the above cell, we have trained a linear regression from a train test split to predict per capita income with the percentage of science and engineering majors in each county as the input. Below, we are making predictions of per capita income given science and engineering major percentage using the train set. This allows us to view our model's predictions of per capita income (h2_y_hat_train) to our _actual_ y values (y_train)

In [ ]:
h2_y_hat_train = h2_model.predict(X_train)

Now our model will use the X_&nbsp;_test_ data to predict y_test. This allows us to determine if the model works on "new" data (data it was not trained on), since the X_test data was not included in X_train. We will compare these predictions (predicted values of county per capita income) to y_test (the actual y values).

In [ ]:
h2_y_hat_test = h2_model.predict(X_test)

To determine how correct the model's predictions are (and how much we should trust it), we will now calculate its evaluation metrics, starting with the train set:

In [ ]:
mse = mean_squared_error(y_train, h2_y_hat_train)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, h2_y_hat_train)
mape = mean_absolute_percentage_error(y_train, h2_y_hat_train)
print('Mean squared error: ' + str(np.round(mse, 3)))
print('Root mean squared error: ' + str(np.round(rmse, 3)))
print('Mean absolute error: ' + str(np.round(mae, 3)))
print('Mean absolute percentage error: ' + str(np.round(mape, 3)))

And now we will do the same for the test set:

In [ ]:
mse = mean_squared_error(y_test, h2_y_hat_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, h2_y_hat_test)
mape = mean_absolute_percentage_error(y_test, h2_y_hat_test)
print('Mean squared error: ' + str(np.round(mse, 3)))
print('Root mean squared error: ' + str(np.round(rmse, 3)))
print('Mean absolute error: ' + str(np.round(mae, 3)))
print('Mean absolute percentage error: ' + str(np.round(mape, 3)))

## Hypothesis 3: Can we reliably predict a county's per capita income based on its percentages of each of the 5 major categories?

## Hypothesis 4: Is there an interaction between the regression variables age and major percentage?

## Data Limitations

- The Census data is limited in that it is essentially a large sample, as it does not cover the entire population, and typically, more vulnerable groups are not represented. 
- This data and any findings from it cannot be generalized to any region outside New York State, as the only counties considered are New York State counties. Especially considering that the state has one of the biggest and most wealthy cities in the world, this is unlikely to be validly mapped to another region. In the same regard, the universities of New York residents are most likely composed of disproportionately more in-state colleges compared to other states, yet another reason to not generalize this data outside of New York. 
- Another limitation is that the Census lumps together a broad variety of majors into "STEM", "Humanities", etc. which may result in a lack of consideration of certain outliers within those major-groups. For example, Psychology may be considered STEM, but it may not follow the same trends as a Computer Science major.
- Another limitation is that the Wikepedia data is outdated by 13 years, as its most recent data is from 2010. This may be an issue because our Census data is much more recent.

## Questions for Reviewers

- How would you utilize graphs and visualizations for our two hypotheses? We were struggling to determine how to use seaborn / plt in our analysis. We feel that there could be room for improvement, as it is short, but we were lost as to what to include.
- How would we better analyze the data / do you have any recommendations for methods to better understand the relationships we are trying to explore? We did a regression, but we weren't too sure what to do after.
- We also were interested in potentially discovering some interaction between the variables, but we did not know how to go about it.
- We want to add a hypothesis 3 discovering whether we can accurately predict a county's per capita income given the percentage of a certain age and major. How would we go about this? We were thinking of a train test split (which we also weren't sure to include in our current analysis), so any feedback on this would be very appreciated!
- How should we interpret / utilize the correlation and covariance matrices? Is there anything that would strengthen our analysis within them?

In the previous model, we discovered from the OLS regression that there is a relatively small relationship between Science and Engineering percentage within a county's graduates on the county's per capita income. Additionally, the p-value for this coefficient is greater than 0.05, indicating that it is likely that the null hypothesis is supported. Thus, we accept the null hypothesis and conclude that it is likely that there is little to no relationship from science and engineering percentage of graduates within a county on that county's income.